In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("crop_yield_dataset.csv")
dataset

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Days_to_Harvest,Yield_tons_per_hectare
0,897.077239,27.676966,False,True,122,6.555816
1,992.673282,18.026142,True,True,140,8.527341
2,147.998025,29.794042,False,False,106,1.127443
3,986.866331,16.644190,False,True,146,6.517573
4,730.379174,31.620687,True,True,110,7.248251
...,...,...,...,...,...,...
999984,302.805345,27.987428,False,False,76,1.347586
999985,932.991383,39.661039,True,False,93,7.311594
999986,867.362046,24.370042,True,False,108,5.763182
999987,492.812857,33.045505,False,False,102,2.070159


In [3]:
dataset.isnull().sum()

Rainfall_mm               0
Temperature_Celsius       0
Fertilizer_Used           0
Irrigation_Used           0
Days_to_Harvest           0
Yield_tons_per_hectare    0
dtype: int64

In [4]:
dataset.columns

Index(['Rainfall_mm', 'Temperature_Celsius', 'Fertilizer_Used',
       'Irrigation_Used', 'Days_to_Harvest', 'Yield_tons_per_hectare'],
      dtype='object')

In [5]:
independent=dataset[['Rainfall_mm', 'Temperature_Celsius', 'Fertilizer_Used','Irrigation_Used', 'Days_to_Harvest']]
independent

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Days_to_Harvest
0,897.077239,27.676966,False,True,122
1,992.673282,18.026142,True,True,140
2,147.998025,29.794042,False,False,106
3,986.866331,16.644190,False,True,146
4,730.379174,31.620687,True,True,110
...,...,...,...,...,...
999984,302.805345,27.987428,False,False,76
999985,932.991383,39.661039,True,False,93
999986,867.362046,24.370042,True,False,108
999987,492.812857,33.045505,False,False,102


In [6]:
dependent=dataset[["Yield_tons_per_hectare"]]
dependent

,Yield_tons_per_hectare
0,6.555816
1,8.527341
2,1.127443
3,6.517573
4,7.248251
...,...
999984,1.347586
999985,7.311594
999986,5.763182
999987,2.070159


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

In [9]:
scy = StandardScaler()
y_train = scy.fit_transform(y_train)
y_test = scy.transform(y_test)


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import time
from sklearn.svm import SVR 
import numpy as np
from sklearn.preprocessing import StandardScaler 

# Generate some sample data for demonstration
independent, dependent = make_regression(n_samples=100, n_features=5, random_state=42)


scaler = StandardScaler()
independent = scaler.fit_transform(independent)

X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.2, random_state=42)

param_grid = {
    'C': [10,100,1000], 
    'kernel': ['linear', 'rbf','poly','sigmoid'],  
    'epsilon': [0.01, 0.1, 1], 
}

grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=3, n_jobs=-1, cv=5)

# Record the start time
start_time = time.time()

# Fit the GridSearchCV object to the training data
grid.fit(X_train, y_train)

# Record the end time
end_time = time.time()

# Calculate the run time
run_time = end_time - start_time

# Print the results
print(f"\nBest parameters found: {grid.best_params_}")
print(f"Best cross-validation score: {grid.best_score_:.4f}")
print(f"GridSearchCV run time: {run_time:.2f} seconds")

# You can now use the best estimator to make predictions
best_svr = grid.best_estimator_
predictions = best_svr.predict(X_test)
print(predictions)

Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameters found: {'C': 100, 'epsilon': 0.01, 'kernel': 'linear'}
Best cross-validation score: 1.0000
GridSearchCV run time: 1.84 seconds
[ 244.59558194 -149.72640632 -261.92511358   53.32699471 -109.01873336
  156.57234268 -145.74497791  115.49494057   52.56362576  271.30578492
  135.86024377  126.71769005 -159.63446867    3.38265228 -221.36190391
   49.63406125   56.32796337   17.25725969  -49.86534605   73.32212036]


In [11]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score) 

The R_score value for best parameter {'C': 100, 'epsilon': 0.01, 'kernel': 'linear'}: 0.9999999986273582


In [12]:
table=pd.DataFrame.from_dict(re)
table  

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011464,0.001582,0.000803,0.000984,10,0.01,linear,"{'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.098031e-09,3
1,0.001986,0.000800,0.000500,0.000777,10,0.01,rbf,"{'C': 10, 'epsilon': 0.01, 'kernel': 'rbf'}",0.262006,0.412341,0.433274,0.131484,0.267245,0.301270,1.107148e-01,31
2,0.001898,0.001052,0.000401,0.000803,10,0.01,poly,"{'C': 10, 'epsilon': 0.01, 'kernel': 'poly'}",0.447253,0.439887,0.489679,0.675824,0.410179,0.492564,9.508701e-02,26
3,0.002046,0.000029,0.000000,0.000000,10,0.01,sigmoid,"{'C': 10, 'epsilon': 0.01, 'kernel': 'sigmoid'}",0.802433,0.797163,0.820679,0.728520,0.778064,0.785372,3.150326e-02,19
4,0.005122,0.002336,0.000752,0.000925,10,0.1,linear,"{'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.238712e-07,6
5,0.000803,0.000984,0.001607,0.000803,10,0.1,rbf,"{'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}",0.262004,0.412140,0.433205,0.131476,0.267231,0.301211,1.106611e-01,32
6,0.000706,0.000403,0.000603,0.000804,10,0.1,poly,"{'C': 10, 'epsilon': 0.1, 'kernel': 'poly'}",0.447212,0.439936,0.489656,0.675941,0.410269,0.492603,9.511533e-02,25
7,0.001589,0.001475,0.000402,0.000803,10,0.1,sigmoid,"{'C': 10, 'epsilon': 0.1, 'kernel': 'sigmoid'}",0.802754,0.796989,0.819388,0.728484,0.778281,0.785179,3.124281e-02,20
8,0.001304,0.001081,0.001139,0.001145,10,1,linear,"{'C': 10, 'epsilon': 1, 'kernel': 'linear'}",0.999960,0.999984,0.999986,0.999963,0.999961,0.999971,1.169608e-05,9
9,0.001174,0.001245,0.000803,0.000984,10,1,rbf,"{'C': 10, 'epsilon': 1, 'kernel': 'rbf'}",0.264167,0.410105,0.432593,0.130317,0.267083,0.300853,1.103271e-01,33


In [13]:
best_svr = grid.best_estimator_

In [14]:
y_pred = best_svr.predict(X_test)


In [15]:
import pickle
filename="finalized_model_SVR.sav"
pickle.dump(best_svr,open(filename,'wb'))

In [16]:
loaded_model=pickle.load(open("finalized_model_SVR.sav",'rb'))
result=loaded_model.predict([[897.077239, 27.676966, 0, 1, 122]])
result

array([58840.72982651])